<a href="https://colab.research.google.com/github/Nithil3007/SciBERT-for-HEP-TH-dataset/blob/main/TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import re
from pandas import *
from sklearn.model_selection import train_test_split

path='/content/drive/MyDrive/Hepth Deep Text/'
paths = []
#Testing the model on papers from 2000-2004
for i in range(2000,2004):
    paths.append(path+str(i)+' hepth deep texts')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
file=pd.read_csv('/content/drive/MyDrive/infPaperTag.csv')
terms=file['0'].values.tolist()
actual=file['2'].values.tolist()
print(paths)

['/content/drive/MyDrive/Hepth Deep Text/2000 hepth deep texts', '/content/drive/MyDrive/Hepth Deep Text/2001 hepth deep texts', '/content/drive/MyDrive/Hepth Deep Text/2002 hepth deep texts', '/content/drive/MyDrive/Hepth Deep Text/2003 hepth deep texts']


In [4]:
texts = []
pokis=[]
labels_to_category = []
for i in range(len(paths)):
    print(i)
    for j in os.listdir(paths[i]):
        if  int(re.findall(r'\d+',j)[0]) in terms:
               with open(paths[i]+"/"+j,encoding="utf8", errors='ignore') as f:
                 temp = f.read()
                 temp = temp.replace("\n"," ").replace('\r','')
                 texts.append(temp)
                 f.close()

0
1
2
3


In [5]:
#Test set 
data_frame = pd.DataFrame({'texts':texts})
len(texts)

8835

In [7]:
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from numpy import array
import keras
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical

In [8]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.7 MB/s 
     |████████████████████████████████| 182 kB 63.9 MB/s 
     |████████████████████████████████| 7.6 MB 30.5 MB/s 


In [10]:
def clean_and_get_tokens(doc):
    tokens = doc.split()
    table = str.maketrans('','',punctuation)  #removes punctuations using 
    tokens = [w.translate(table) for w in tokens] #dictionary of punctuations
    tokens = [word for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    tokens = [word for word in tokens if len(word)>2]
    return tokens

In [9]:
words = set()
vocab = {}
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
token = data_frame['texts'][0].split()
print(data_frame['texts'][0])
table = str.maketrans('','',punctuation)
tokens = [w.translate(table) for w in token] 
print(tokens)
tokens = [word for word in tokens if word.isalpha()]
stop_words = set(stopwords.words('english'))
tokens = [w for w in tokens if not w in stop_words]
tokens = [word for word in tokens if len(word)>2]
print(tokens)

-------------------------------- E:/BigData/Project/Run/Files_Keywords/TextFiles\0006174v1.txt -------------------------------- (6), and the dynamical equation for , the complete behaviour of our model is similar  to the friedmann cosmology.in  the original einsteins proposal the cosmological term was exactly a constant introduced  by hand in einstein hilbert lagrangean as a kind of force opposite to that of gravitation.in accordance with [2] we choose  = (a), where a means the scale factor for  friedmann-robertson-walker universe.on taking  = 1 the following solutions is found.fcpnunes@cce.ufes.br/maria@gbl.com.br  khanna@phys.ualberta.ca  2  introduction  in the last few years several paper [1, 5] have appeared discussing the cosmological  constant problem when that term is considered as part of the content of the universe.the analyse of this problem with  = bam for m 6= 2 is presently being considered for the same initial hypothesis and the equation of state will be obtained.in the

In [12]:
documents = data_frame['texts']
for doc in documents:
    tokens = clean_and_get_tokens(doc)
    for token in tokens:
        if token in vocab:
            vocab[token] += 1
        else:
            vocab[token] = 1

for word in vocab:
    if vocab[word] > 5:
        words.add(word)

In [13]:
train_documents = []
train_data=data_frame
for doc in train_data['texts']:
    tokens = doc.split()
    final_tokens = []
    
    for token in tokens:
        if token in words:
            final_tokens.append(token)
    final_string = ' '.join(final_tokens)
    train_documents.append(final_string)
print(train_documents[0])

dynamical equation complete behaviour model similar friedmann original einsteins proposal cosmological term exactly constant introduced hand einstein hilbert lagrangean kind force opposite accordance choose means scale factor taking following solutions introduction last years several paper appeared discussing cosmological constant problem term considered part content analyse problem presently considered initial hypothesis equation state einsteins equation cosmological term appears left side meaning must general solutions equation dynamic equation scale factor solutions particular values clear possibilities origin time exactly way standard cosmology cosmological term solutions found almost space particular hypothesis numerically approximate line element fact cosmological term constant function one show equation governs behavior scale factor presence cosmological constant like original einstein model given plot end paper conclusions comments found direct solutions non linear dierential e

In [14]:
#tokenizer = Tokenizer()
#tokenizer.fit_on_texts(train_documents)
#encoded_docs = tokenizer.texts_to_sequences(train_documents)
from transformers import AutoTokenizer

tz = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
for i in range (0,len(train_documents)):
  tz.tokenize(train_documents[i])
encoded_docs = []
for i in range(0,len(train_documents)):
  encoded_docs.append(tz.convert_tokens_to_ids(tz.tokenize(train_documents[i])))

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/228k [00:00<?, ?B/s]

In [15]:
max_length = max(([len(s.split()) for s in train_documents]))
Xtrain = pad_sequences(encoded_docs, maxlen=745, padding='post')
print(Xtrain)

[[10245  2037  2467 ...     0     0     0]
 [  993 12829  1532 ...     0     0     0]
 [ 3790  3676 18916 ...     0     0     0]
 ...
 [ 1196  4001 15811 ...     0     0     0]
 [  959  7176  1245 ...     0     0     0]
 [ 1532  1157   585 ...     0     0     0]]


In [17]:
from tensorflow import keras
model = keras.models.load_model('/content/drive/MyDrive/trained models/Train.h5')

In [18]:
import tensorflow as tf
sess=tf.compat.v1.InteractiveSession()

predictions=model.predict(Xtrain)

indexes = tf.argmax(predictions, axis=1)
answers=indexes.numpy()

277/277 [==============================] - 13s 46ms/step


In [19]:
res = pd.DataFrame(answers)
#res.index = pokis # its important for comparison
res.columns = ["prediction"]
#res.to_csv("prediction1994-2003.csv")

In [20]:
test=[]
for i in range(len(paths)):
    
    print(paths[i])
    for j in os.listdir(paths[i]):
        x=int(re.findall(r'\d+',j)[0])
        if x in terms:
          indx=terms.index(x)
          test.append([x,actual[indx]])

/content/drive/MyDrive/Hepth Deep Text/2000 hepth deep texts
/content/drive/MyDrive/Hepth Deep Text/2001 hepth deep texts
/content/drive/MyDrive/Hepth Deep Text/2002 hepth deep texts
/content/drive/MyDrive/Hepth Deep Text/2003 hepth deep texts


In [21]:
final=[]
yact,ypred=[],[]
for a,b in zip(test,answers):
  yact.append(a[1])
  ypred.append(b)
  final.append([a[0],b,a[1]])
print(final[1])

[6175, 0, 0]


In [22]:
from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1', ]
print(classification_report(yact, ypred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.74      0.85      0.79      6582
     class 1       0.24      0.14      0.18      2253

    accuracy                           0.67      8835
   macro avg       0.49      0.50      0.49      8835
weighted avg       0.62      0.67      0.64      8835



In [23]:
final=pd.DataFrame(final)
print(final.head())
final.columns=['FileName','Predicted','Actual']
final.to_csv('RESULTS.csv',index=None)

      0  1  2
0  6174  0  0
1  6175  0  0
2  6176  0  0
3  6177  0  0
4  6178  0  0
